In [1]:
import spacy
import numpy as np
import pandas as pd
import os, sys, time

cwd = os.getcwd()
frameworkDir = os.path.abspath(os.path.join(cwd, os.pardir, 'src'))
sys.path.append(frameworkDir)

from dackar.pipelines.CustomPipelineComponents import aliasResolver
from dackar.utils.nlp.nlp_utils import resetPipeline

nlp = spacy.load("en_core_web_lg")
# import logging
# logger = logging.getLogger(__name__)

# ch = logging.StreamHandler()
# logger.addHandler(ch)

Warming up PyWSD (takes ~10 secs)... took 3.738736867904663 secs.
/Users/wangc/miniconda3/envs/nlp_libs/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [2]:
# aliasResolver pipeline should always after "entity_ruler"
pipelines = ['aliasResolver']
resetPipeline(nlp, pipelines)
print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fa417e77ea0>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7fa45fa5fef0>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fa4139d3940>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fa417e8f2c0>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7fa415217040>), ('aliasResolver', <function aliasResolver at 0x7fa4185c7f70>)]


In [3]:
text="1-71120-P1, CLEAN ZEBRA MUSSELS IN PUMPHOUSE. 1-71120-PM1 REQUIRES OIL TOP UP. 71120, CLEAN TRASH RACKS FROM FOREBAY SIDE"

In [4]:
# Load module from DACKAR
from dackar.pipelines.GeneralEntity import GeneralEntity
from spacy import displacy
# Load pattern generation from DACKAR
from dackar.utils.nlp.nlp_utils import generatePatternList

pipeline = [pipe for (pipe,_) in nlp.pipeline]
if "enity_ruler" in pipeline:
    nlp.remove_pipe("entity_ruler")
if "ner" in pipeline:
    nlp.remove_pipe("ner")

# Specify Entities Labels and IDs
entLabel = "cws_component"        # user defined entity label
entId = "OPM"  
entIDList = ['1-71120-P1', '1-71120-PM1', '71120', 'FOREBAY']
patternsEnts = generatePatternList(entIDList, label=entLabel,    id=entId,    nlp=nlp, attr="LEMMA")

generalEntity = GeneralEntity(nlp, patternsEnts)

doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

In [5]:
for ent in doc.ents:
    print(ent.text, ent._.alias)

1-71120-P1 cw pump
1-71120-PM1 cw p1 pump motor
71120 ccw
FOREBAY None
